<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please **DO NOT** use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [8]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



The latest news on Amazon highlights significant advances in their AI technology focused on improving delivery accuracy, product demand prediction, and robotics. Key points include:

1. Amazon is using generative AI technology called Wellspring to enhance delivery location accuracy.
2. They have developed an AI-powered demand forecasting model that predicts what products customers want, where, and when.
3. An agentic AI team is enabling robots to understand natural language commands, improving their operational efficiency.
4. Amazon recently deployed its one millionth robot, powered by a new AI foundation model that enhances the robotic fleet's capabilities.
5. These AI advancements aim to benefit both customers and employees by improving delivery precision, forecasting accuracy, and robotic automation, especially during peak shopping seasons like Black Friday and Cyber Monday.

This shows Amazon's continued commitment to leveraging cutting-edge AI to optimize logistics and customer experience.

If you want, I can keep monitoring for more updates or specific financial news about Amazon.

### Look at the trace

https://platform.openai.com/traces

In [9]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-03 20:50:01", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [10]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [11]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Ed and your account is under your name, Ed.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-03 20:50:01", 10000.0], ["2025-07-03 20:50:02", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please

### And to run our Trader

In [12]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Summary of Actions Taken:

1. **Market Research Conducted**: 
   - Notable news included strong job data affecting Fed rate cut expectations and an activist stake in Tripadvisor.
   - Genmab was noted for a neutral stance from analysts, indicating potential for recovery.

2. **Stock Trading Decisions**:
   - **Bought**: 
     - **Genmab (GMAB)**: 50 shares at approximately $20.48 each.
       - *Rationale*: Market undervaluation and neutral stance by Morgan Stanley suggest potential for short-term gains.
   - **Sell Attempted**: 
     - **JPMorgan Chase (JPM)**: Attempted to sell 20 shares but was unsuccessful due to not holding any shares in the position.
   
3. **Current Portfolio Overview**:
   - **New Holdings**: 50 shares of GMAB.
   - **Current Balance**: $8,975.96.
   - **Total Portfolio Value**: $9,997.96.
   - **Total Profit/Loss**: -$2.04 due to trading transaction costs.

### Next Steps:
- Monitor GMAB for price changes and consider potential selling opportunities based on performance and news updates.
- Re-evaluate the portfolio for future trades based on emerging market conditions and news.

### Then go and look at the trace

http://platform.openai.com/traces


In [13]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "ed", "balance": 8975.956, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"GMAB": 50}, "transactions": [{"symbol": "GMAB", "quantity": 50, "price": 20.480880000000003, "timestamp": "2025-07-03 20:51:13", "rationale": "Market undervaluation and neutral stance by Morgan Stanley suggest potential for short-term gains."}], "portfolio_value_time_series": [["2025-07-03 20:50:01", 10000.0], ["2025-07-03 20:50:02", 10000.0], ["2025-07-03 20:51:13", 9997.956], ["2025-07-03 20:51:18", 9997.956]], "total_portfolio_value": 9997.956, "total_profit_loss": -2.043999999999869}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this **async with AsyncExitStack() as stack**:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [14]:
from traders import Trader


In [15]:
trader = Trader("Ed")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

In [ ]:
for params in all_params:
    print(f"Params: {params['command']} {params['args']}")

In [ ]:
for tool in mcp_tools:
    print(tool)